In [ ]:
!pip install -U sentence-transformers
!pip3 install datasets
!pip3 install rouge_score
!pip3 install git+https://github.com/huggingface/transformers
!pip3 install sentencepiece

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 2.3MB 7.2MB/s 
     |████████████████████████████████| 1.2MB 26.6MB/s 
     |████████████████████████████████| 3.3MB 40.8MB/s 
     |████████████████████████████████| 901kB 40.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=81a4540bf27cd386a1300132d15fa456a08551ce204f1ac3621db46efeaaa0d8
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers
     |████████████████████████████████| 245kB 6.7MB/s 
     |████████████████████████████████| 245kB 34.9MB/s 
     |████████████████████████████████| 122kB 30.1MB/s 
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-s82ia3qz
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-s82ia3qz
  Installing build dependencies ... do

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/allclean/finalfulldata_sections.csv" "train.csv"

from datasets import load_dataset
import pandas as pd
train=pd.read_csv('/content/train.csv')
train.shape

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(10864, 9)

In [ ]:
!cp "/content/drive/My Drive/allclean/conditioned_data.csv" "train.csv"
from datasets import load_dataset
import pandas as pd
train=pd.read_csv('/content/train.csv')
train.shape

(10589, 12)

In [ ]:
train.columns

Index(['Unnamed: 0', 'title', 'abstract', 'articles', 'wordcount_articles',
       'wordcount_abstract', 'Result', 'Introduction', 'section_names',
       'condition_wordcount', 'extractedsam', 'conditioned'],
      dtype='object')

In [ ]:
train['conditioned'].head()

0    Introduction\nCanine coronavirus  was first is...
1    Introduction\nT he 2 0  5 0  oligoadenylate sy...
2    Introduction\nThe major histocompatibility com...
3    Introduction\nThe Coronavirinae, which is subd...
4    Introduction\nCoronaviruses are positive stran...
Name: conditioned, dtype: object

In [ ]:
train=train.reset_index(drop=True)

In [ ]:
def get_wordcounts(x):
    length = len(str(x).split())
    return length

In [ ]:
train_x=train[train['wordcount_articles']<4000]
train_x=train_x[train_x['wordcount_articles']>2500]

In [ ]:
train_x.shape

(4904, 12)

In [ ]:
train_x=train_x.reset_index(drop=True)

In [ ]:
for i in range(len(train)):
  x=train.iloc[i,3]
  x=get_wordcounts(x)
  if(x>4500):
    print(i)
    print(x)
    break




3
4627


In [ ]:
test1 = train.iloc[[3],:] 

In [ ]:
test_y=train[['articles', 'abstract']]
test_x=train[['conditioned', 'abstract']]
est_x.rename(columns = {'conditioned':'articles'}, inplace = True)

In [ ]:
test_x=train[['conditioned', 'abstract']]

In [ ]:
test_x.columns, test_y.columns

(Index(['articles', 'abstract'], dtype='object'),
 Index(['articles', 'abstract'], dtype='object'))

In [ ]:
test_x.head(2)

,articles,abstract
0,"Introduction\nThe Coronavirinae, which is subd...",Abstract\n\nIt has been demonstrated that the ...
1,Introduction\nCoronaviruses are positive stran...,Abstract\n\nThe human coronavirus NL63 is gene...


In [ ]:
test1['wordcount_articles']

3    4684
Name: wordcount_articles, dtype: int64

In [ ]:
test_y.head(2)

,articles,abstract
0,"Introduction The Coronavirinae, which is subd...",Abstract\n\nIt has been demonstrated that the ...
1,Introduction Coronaviruses are positive stran...,Abstract\n\nThe human coronavirus NL63 is gene...


In [ ]:
test_x.rename(columns = {'conditioned':'articles'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#val['articles'] = val['articles'].apply(lambda x: finalclean(x))
test1['abstract'] = test1['abstract'].apply(lambda x: removeurls(x))
#val['abstract'] = val['abstract'].apply(lambda x: removeurls(x))
test1['abstract'] = test1['abstract'].apply(lambda x: removbrackets(x))
#val['abstract'] = val['abstract'].apply(lambda x: removbrackets(x))
test1['abstract'] = test1['abstract'].apply(lambda x: misc_clean(x))
#val['abstract'] = val['abstract'].apply(lambda x: misc_clean(x))
test1['abstract'] = test1['abstract'].apply(lambda x: finalclean(x))
#val['abstract'] = val['abstract'].apply(lambda x: finalclean(x))
test1['abstract'] = test1['abstract'].apply(lambda x: single(x))
test1['abstract'] = test1['abstract'].apply(lambda x: removelines(x))

In [ ]:
from datasets import Dataset
testx = Dataset.from_pandas(test_x)
testy = Dataset.from_pandas(test_y)

In [ ]:
% cd drive/My\ Drive/

/content/drive/My Drive


##Loading trained BIGBIRD standalone model

In [ ]:
DATASET_NAME = "pubmed"
device = "cuda"

#MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
MODEL_ID = f"BigBirdBeast"
from transformers import AutoTokenizer, AutoModelForMaskedLM
  

from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import PegasusTokenizer, BigBirdPegasusModel
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
model = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [ ]:
DATASET_NAME = "pubmed"
device = "cuda"

#MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
MODEL_ID = f"model_X"
from transformers import AutoTokenizer, AutoModelForMaskedLM
  

from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import PegasusTokenizer, BigBirdPegasusModel
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
model1 = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

##Loading trained BIGBIRD with extraction layer

In [ ]:
DATASET_NAME = "pubmed"
device = "cuda"

#MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
MODEL_ID = f"model_C"
from transformers import AutoTokenizer, AutoModelForMaskedLM
  

from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import PegasusTokenizer, BigBirdPegasusModel
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
model2 = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [ ]:
def generate_answer(batch):
    inputs_dict = tokenizer(batch["articles"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch
def generate_answer1(batch):
    inputs_dict = tokenizer(batch["articles"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model1.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch
def generate_answer2(batch):
    inputs_dict = tokenizer(batch["articles"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model2.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch

In [ ]:
from datasets import load_dataset, load_metric
rouge = load_metric("rouge")

In [ ]:
train.shape

(10589, 12)

In [ ]:
train.columns

Index(['Unnamed: 0', 'title', 'abstract', 'articles', 'wordcount_articles',
       'wordcount_abstract', 'Result', 'Introduction', 'section_names',
       'condition_wordcount', 'extractedsam', 'conditioned'],
      dtype='object')

In [ ]:
train.iloc[1573]

In [ ]:
teet=titoo.iloc[0,3]

In [ ]:
titoo.shape

(1, 2)

###### ROUGE score of article length greater than 4000 words- BigBIRD standalone

In [ ]:

dataset_small = testy.select(range(2))
result_small = dataset_small.map(generate_answer1)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The poly tail length of bovine coronavirus, which is a member of the alphacoronavirus family, is relatively short in infected cells at 0 2 postinfection, increases to a peak length at 6 10 hpi, and gradually decreases in size after 10 of infection. In addition, the length of the coronaviral poly tail is determined to be associated with the efficiency of virus translation. In this study, the regulation of coronaviral poly tail length during infection of chicken embryo kidney cells was investigated. The poly tail length was determined by head to tail ligation, and the lengths of the coronaviral poly tail were determined to be 33 nt, 62 nt, 51 nt, 41 nt, and 40 nt at 1, 2, 8, 12, 24, 36, and 48 hpi, respectively. The poly tail length was also determined to be reduced in comparison with that from inoculum used for infection. The results demonstrated that the regulation of coronaviral poly tail length during infection occurs not only in cell cultures but also in living systems and 

{'rouge1': AggregateScore(low=Score(precision=0.3288590604026846, recall=0.3161290322580645, fmeasure=0.3223684210526316), mid=Score(precision=0.4639464384139027, recall=0.40507646832026734, fmeasure=0.4319265686049184), high=Score(precision=0.5990338164251208, recall=0.4940239043824701, fmeasure=0.5414847161572053)),
 'rouge2': AggregateScore(low=Score(precision=0.08108108108108109, recall=0.07792207792207792, fmeasure=0.07947019867549669), mid=Score(precision=0.17160850170558908, recall=0.14696103896103896, fmeasure=0.1581561519693273), high=Score(precision=0.2621359223300971, recall=0.216, fmeasure=0.2368421052631579)),
 'rougeL': AggregateScore(low=Score(precision=0.16778523489932887, recall=0.16129032258064516, fmeasure=0.16447368421052633), mid=Score(precision=0.26022111986512336, recall=0.2260634879835497, fmeasure=0.2416254883934728), high=Score(precision=0.3526570048309179, recall=0.2908366533864542, fmeasure=0.3187772925764193)),
 'rougeLsum': AggregateScore(low=Score(precisi

###### ROUGE score of article length greater than 4000 words- BigBIRD with extraction layer

In [ ]:

dataset_small = testx.select(range(2))
result_small = dataset_small.map(generate_answer2)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract The regulation of coronaviral poly tail length is a common feature of coronaviruses and can occur in other cell types and living systems other than human rectal tumor 18 cells. In this study, we demonstrate that the regulation of coronaviral poly tail length is also regulated in noncancerous cells, primary cell cultures, and a living system, suggesting that the regulation may be a common feature of coronaviruses and is not a host cell specific. The length of the coronaviral poly tail was increased at 4 and 8 hpi during the early stage of infection in noncancerous cells, primary cell cultures, and a living system, suggesting that the regulation is not a host cell specific. The length of the coronaviral poly tail was increased again after 5 days of infection in the same noncancerous cells, primary cell cultures, and a living system, suggesting that the length of the coronaviral poly tail is increased again after 5 days of infection in the same noncancerous cells, primary cell cu

{'rouge1': AggregateScore(low=Score(precision=0.5852534562211982, recall=0.3225806451612903, fmeasure=0.4347826086956521), mid=Score(precision=0.6259600614439325, recall=0.41427837038941007, fmeasure=0.48875882571534746), high=Score(precision=0.6666666666666666, recall=0.5059760956175299, fmeasure=0.5427350427350428)),
 'rouge2': AggregateScore(low=Score(precision=0.22972972972972974, recall=0.11038961038961038, fmeasure=0.14912280701754385), mid=Score(precision=0.2306056056056056, recall=0.1551948051948052, fmeasure=0.18185754084782774), high=Score(precision=0.23148148148148148, recall=0.2, fmeasure=0.21459227467811162)),
 'rougeL': AggregateScore(low=Score(precision=0.33640552995391704, recall=0.2, fmeasure=0.26956521739130435), mid=Score(precision=0.3748694316436252, recall=0.2454183266932271, fmeasure=0.29076551467855816), high=Score(precision=0.41333333333333333, recall=0.2908366533864542, fmeasure=0.31196581196581197)),
 'rougeLsum': AggregateScore(low=Score(precision=0.336405529

In [ ]:
train=train[train['wordcount_articles']>4500]

In [ ]:
sexy=test_y.iloc[0,0]

In [ ]:
sexy=get_wordcounts(sexy)
sexy

4627

In [ ]:
test_yy=train_x[['articles', 'abstract']]
test_xx=train_x[['conditioned', 'abstract']]
test_xx.rename(columns = {'conditioned':'articles'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
test_xx=test_xx.reset_index(drop=True)
test_yy=test_yy.reset_index(drop=True)

In [ ]:
from datasets import Dataset
testxx = Dataset.from_pandas(test_xx)
testyy = Dataset.from_pandas(test_yy)
testxx, testyy

(Dataset({
     features: ['articles', 'abstract'],
     num_rows: 4904
 }), Dataset({
     features: ['articles', 'abstract'],
     num_rows: 4904
 }))

In [ ]:
z=test_yy.iloc[0,0]
z1=test_yy.iloc[1,0]
z2=test_yy.iloc[2,0]

In [ ]:
z=get_wordcounts(z)
z1=get_wordcounts(z1)
z2=get_wordcounts(z2)
z, z1, z2

(2907, 3638, 3686)

In [ ]:
test_x1=test_xx[1:3]
test_y1=test_yy[1:3]

In [ ]:
from datasets import Dataset
testx1 = Dataset.from_pandas(test_x1)
testy1 = Dataset.from_pandas(test_y1)
testx1, testy1

(Dataset({
     features: ['articles', 'abstract'],
     num_rows: 2
 }), Dataset({
     features: ['articles', 'abstract'],
     num_rows: 2
 }))

In [ ]:
from datasets import Dataset
teet = Dataset.from_pandas(test_x1)
testy1 = Dataset.from_pandas(test_y1)
testx1, testy1

In [ ]:
bittiy=trainless[['articles','abstract']]
bittix=trainless[['conditioned', 'abstract']]

In [ ]:
bittix.rename(columns = {'conditioned':'articles'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
from datasets import Dataset
bity = Dataset.from_pandas(bittiy)
bitx = Dataset.from_pandas(bittix)

In [ ]:
bity, bitx

(Dataset({
     features: ['articles', 'abstract', '__index_level_0__'],
     num_rows: 476
 }), Dataset({
     features: ['articles', 'abstract', '__index_level_0__'],
     num_rows: 476
 }))

##ROUGE Scores of Short Articles (<2000 words) - BIGBIRD standalone

In [ ]:
dataset_small = bity.select(range(3))
result_small = dataset_small.map(generate_answer1)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract We report the case of a 7year old boy who presented with prolonged fever, cough, and chest rays showing a welldemarcated round mass measuring 5.9 5.6 4.3 cm in the left lower lung field, findings which were typical for round pneumonia. The urinary pneumococcal antigen test was positive, and serum anti SARScoplasma pneumoniae antibody titer measurement using a microparticle agglutination method was 1 160. The oral administration of antibiotics including azithromycin and, which was subsequently replaced by ceftibuten due to moderate diarrhea, the fever subsided 2 days later and the round patch had completely resolved on the 18th day after the diagnosis. This evidence suggests treating classical round pneumonia with antibiotics first and waiving unwarranted advanced imaging studies, while alternative etiologies such as abscesses, tuberculosis, nonbacterial infections, congenital malformations, or neoplasms should still be considered in patients with atypical features or poor trea

{'rouge1': AggregateScore(low=Score(precision=0.2553191489361702, recall=0.24324324324324328, fmeasure=0.3380281690140845), mid=Score(precision=0.6064881543604947, recall=0.5271311774201948, fmeasure=0.5283221368910075), high=Score(precision=0.9797297297297297, recall=0.838150289017341, fmeasure=0.9034267912772586)),
 'rouge2': AggregateScore(low=Score(precision=0.0588235294117647, recall=0.11578947368421051, fmeasure=0.07801418439716312), mid=Score(precision=0.4399145623161546, recall=0.34939022226952615, fmeasure=0.37641868199461737), high=Score(precision=0.9319727891156463, recall=0.7965116279069767, fmeasure=0.8589341692789968)),
 'rougeL': AggregateScore(low=Score(precision=0.16489361702127658, recall=0.15135135135135136, fmeasure=0.2137404580152672), mid=Score(precision=0.50275323679579, recall=0.43747276901178633, fmeasure=0.44515903614915175), high=Score(precision=0.9797297297297297, recall=0.838150289017341, fmeasure=0.9034267912772586)),
 'rougeLsum': AggregateScore(low=Score

In [ ]:
dataset_small = bity.select(range(6))
result_small = dataset_small.map(generate_answer1)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract We report the case of a 7year old boy who presented with prolonged fever, cough, and chest rays showing a welldemarcated round mass measuring 5.9 5.6 4.3 cm in the left lower lung field, findings which were typical for round pneumonia. The urinary pneumococcal antigen test was positive, and serum anti SARScoplasma pneumoniae antibody titer measurement using a microparticle agglutination method was 1 160. The oral administration of antibiotics including azithromycin and, which was subsequently replaced by ceftibuten due to moderate diarrhea, the fever subsided 2 days later and the round patch had completely resolved on the 18th day after the diagnosis. This evidence suggests treating classical round pneumonia with antibiotics first and waiving unwarranted advanced imaging studies, while alternative etiologies such as abscesses, tuberculosis, nonbacterial infections, congenital malformations, or neoplasms should still be considered in patients with atypical features or poor trea

{'rouge1': AggregateScore(low=Score(precision=0.232708780581121, recall=0.2938469736822663, fmeasure=0.23903801503118785), mid=Score(precision=0.4695417674141078, recall=0.44898140075559434, fmeasure=0.39324993546540526), high=Score(precision=0.7182990323234226, recall=0.6398877603129887, fmeasure=0.6155186692059232)),
 'rouge2': AggregateScore(low=Score(precision=0.10009682885499878, recall=0.11728198439940431, fmeasure=0.09705050694761194), mid=Score(precision=0.2900323872238954, recall=0.2713832600785511, fmeasure=0.24989996731088582), high=Score(precision=0.5703194858190189, recall=0.5119985040119679, fmeasure=0.510530290468278)),
 'rougeL': AggregateScore(low=Score(precision=0.18101196232845168, recall=0.20480084863773018, fmeasure=0.1686078542606252), mid=Score(precision=0.35441118977704344, recall=0.37836892916099113, fmeasure=0.31849621399127176), high=Score(precision=0.6274163377821914, recall=0.5777733601775241, fmeasure=0.5634336814047165)),
 'rougeLsum': AggregateScore(low=

#Evaluation results BIGBIRD standalone trained model##

In [ ]:
dataset_small = bity.select(range(4))
result_small = dataset_small.map(generate_answer1)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract We report the case of a 7year old boy who presented with prolonged fever, cough, and chest rays showing a welldemarcated round mass measuring 5.9 5.6 4.3 cm in the left lower lung field, findings which were typical for round pneumonia. The urinary pneumococcal antigen test was positive, and serum anti SARScoplasma pneumoniae antibody titer measurement using a microparticle agglutination method was 1 160. The oral administration of antibiotics including azithromycin and, which was subsequently replaced by ceftibuten due to moderate diarrhea, the fever subsided 2 days later and the round patch had completely resolved on the 18th day after the diagnosis. This evidence suggests treating classical round pneumonia with antibiotics first and waiving unwarranted advanced imaging studies, while alternative etiologies such as abscesses, tuberculosis, nonbacterial infections, congenital malformations, or neoplasms should still be considered in patients with atypical features or poor trea

{'rouge1': AggregateScore(low=Score(precision=0.35307472755578617, recall=0.21105474118186018, fmeasure=0.2967531941604066), mid=Score(precision=0.6164514816240296, recall=0.4402636373024342, fmeasure=0.4665333798565845), high=Score(precision=0.8963826631509558, recall=0.6894235275738165, fmeasure=0.7620771357114651)),
 'rouge2': AggregateScore(low=Score(precision=0.1263544891640867, recall=0.08243677905944986, fmeasure=0.08967375886524823), mid=Score(precision=0.38857789704575785, recall=0.2781991292871786, fmeasure=0.3076473448292964), high=Score(precision=0.7576215671453767, recall=0.6263310893512851, fmeasure=0.6695339602925809)),
 'rougeL': AggregateScore(low=Score(precision=0.21457930367504835, recall=0.1299129638112689, fmeasure=0.18189841969589418), mid=Score(precision=0.4746259032065986, recall=0.34450402705659516, fmeasure=0.3763095954142511), high=Score(precision=0.8257063882063882, recall=0.6664505546008437, fmeasure=0.7310052079617607)),
 'rougeLsum': AggregateScore(low=Sc

# Evaluation results BIGBIRD with extraction model trained model#

In [ ]:
dataset_small = bitx.select(range(6))
result_small = dataset_small.map(generate_answer2)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

Abstract A child may develop an opaque pulmonary consolidation with unusually round shape, which raises the concern of a tumor in the chest, causing anxiety to the pediatrician and parents. We report the case of a child with so called round pneumonia, whose initial chest computed tomography findings mimicked those of a lung mass, but subsequently showed complete response to oral antibiotics alone. The findings and recommendations in the literature are also reviewed.
Abstract The crazy paving sign is an area of scattered or diffuse ground glass attenuation with superimposed interlobular septal thickening and intralobular lines. The crazy paving sign has a variety of causes, including infectious, neoplastic, idiopathic, inhalational, and hemorrhagic disorders. In this report, we describe the crazy paving sign in a 69 year old woman with parainfluenza virus pneumonia in a non immunocompromised host.
Abstract The aim of this study was to evaluate a multiplex amplification and suspension ar

{'rouge1': AggregateScore(low=Score(precision=0.13039597372531836, recall=0.2542117801155716, fmeasure=0.15571714995361643), mid=Score(precision=0.30352930862924693, recall=0.33580139665544373, fmeasure=0.23778940514328348), high=Score(precision=0.4885706769210159, recall=0.4343149389137633, fmeasure=0.3086754270174341)),
 'rouge2': AggregateScore(low=Score(precision=0.04424584484299022, recall=0.08410075576538245, fmeasure=0.05404682004198554), mid=Score(precision=0.11945540432221263, recall=0.14132437810062737, fmeasure=0.09340086502511007), high=Score(precision=0.24138707088944053, recall=0.21513191947259455, fmeasure=0.1463634236238814)),
 'rougeL': AggregateScore(low=Score(precision=0.0991141536879048, recall=0.16829017803685922, fmeasure=0.11952814811130681), mid=Score(precision=0.2023218259816173, recall=0.2503234816281563, fmeasure=0.16256915904883257), high=Score(precision=0.32978560967082426, recall=0.3390457321471964, fmeasure=0.2126302269091667)),
 'rougeLsum': AggregateSco

In [ ]:
from rouge import Rouge
rouge = Rouge()

In [ ]:
!pip install rouge